In [1]:
import cv2
import numpy as np
import pickle
import pandas as pd
from datetime import datetime
from keras_facenet import FaceNet
import tkinter as tk
import os

# Load trained model and label encoder
with open("knn_model.pkl", "rb") as f:
    knn = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Load FaceNet model
embedder = FaceNet()

# Dictionary to track attendance with entry and exit times
attendance_log = {}
recognition_counter = {}

# Load attendance.csv to map Person IDs to Names, Roll No, and Department
attendance_df = pd.read_csv("Attendance.csv")
person_info = {}
for _, row in attendance_df.iterrows():
    person_id_str = str(row["Person ID"]).zfill(3)
    person_info[person_id_str] = {
        "Name": row["Name"],
        "Roll No": row["Roll No"],
        "Department": row["Department"]
    }

# Create folder for unknown snapshots
unknown_folder = "unknown_snapshots"
if not os.path.exists(unknown_folder):
    os.makedirs(unknown_folder)

# Open webcam
cap = cv2.VideoCapture(0)

# Flag to stop camera
stop_camera = False

# Confidence threshold
CONFIDENCE_THRESHOLD = 0.9

# Flag to prevent multiple snapshots of the same unknown face
snapshot_taken = False

# Function to stop camera
def stop():
    global stop_camera
    stop_camera = True

# Create Tkinter window
root = tk.Tk()
root.title("Attendance System")
stop_button = tk.Button(root, text="Stop Camera", command=stop, font=("Arial", 12), bg="red", fg="white")
stop_button.pack(pady=10)

while not stop_camera:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame from webcam.")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    faces = embedder.extract(rgb_frame, threshold=0.50)

    for face in faces:
        embedding = face["embedding"]
        embedding = np.array(embedding).reshape(1, -1)

        # Predict person with probability
        pred = knn.predict(embedding)
        pred_proba = knn.predict_proba(embedding)
        confidence = np.max(pred_proba)

        # Process recognition
        person_id = str(pred[0]).zfill(3)
        info = person_info.get(person_id, {"Name": "Unknown", "Roll No": "N/A", "Department": "N/A"})
        name = info["Name"]

        # Only process as known face if confidence is high and person is in dataset
        if confidence >= CONFIDENCE_THRESHOLD and name != "Unknown":
            # Known face processing (mark attendance and green box)
            color = (0, 255, 0)  # Green for known faces
            
            # Stabilization counter
            if name not in recognition_counter:
                recognition_counter[name] = 1
            else:
                recognition_counter[name] += 1
            
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # Log entry time (first detection)
            if recognition_counter[name] == 5 and name not in attendance_log:
                attendance_log[name] = {
                    "Name": name,
                    "Roll No": info["Roll No"],
                    "Department": info["Department"],
                    "Entry Time": current_time,
                    "Exit Time": None
                }
            
            # Update exit time (continuous detection)
            if name in attendance_log and recognition_counter[name] >= 5:
                attendance_log[name]["Exit Time"] = current_time

            # Draw green bounding box and name
            x, y, w, h = face["box"]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # Reset snapshot flag when known face is detected
            snapshot_taken = False
        else:
            # Unknown face processing (save one snapshot with red box, timestamp, and draw red box)
            recognition_counter.clear()  # Reset counter
            
            # Draw red bounding box and "Unknown" label on the frame
            x, y, w, h = face["box"]
            color = (0, 0, 255)  # Red for unknown faces
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # Save snapshot only if not already taken
            if not snapshot_taken:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                timestamp_display = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                
                # Create a copy of the frame with red box and "Unknown" label
                snapshot_frame = frame.copy()
                # Add timestamp text to the snapshot
                cv2.putText(snapshot_frame, timestamp_display, (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                # Save the snapshot with red box, "Unknown", and timestamp
                snapshot_path = os.path.join(unknown_folder, f"unknown_{timestamp}.jpg")
                cv2.imwrite(snapshot_path, snapshot_frame)
                print(f"Snapshot of unknown person saved: {snapshot_path}")

                # Set flag to prevent multiple snapshots
                snapshot_taken = True

    cv2.imshow("Attendance System", frame)
    root.update()

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
root.destroy()

# Save attendance log for known faces
if attendance_log:
    df_attendance = pd.DataFrame.from_dict(attendance_log, orient="index")
    df_attendance = df_attendance[["Name", "Roll No", "Department", "Entry Time", "Exit Time"]]
    df_attendance.to_csv("Attendance_Report.csv", index=False)
    print("✅ Attendance recorded and saved as 'Attendance_Report.csv'.")
else:
    print("No recognized faces detected with sufficient confidence.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Snapshot of unknown person saved: unknown_snapshots\unknown_20250418_215453.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Snapshot of unknown person saved: unknown_snapshots\unknown_20250418_215453.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Snapshot of unknown person saved: unknown_snapshots\unknown_20250418_215455.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Snapshot of unknown person saved: unknown_snapshots\unknown_20250418_215459.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Snapshot of unknown person saved: unknown_snapshots\unknown_20250418_215500.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Snapshot of unknown person saved: